In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [ ]:
file_path = "Resources/state_M2017_dl.xlsx"
df = pd.read_excel(file_path, index = False)
df

In [ ]:
new_df = df[df["OCC_CODE"].str.contains("^15")]
new_df

In [ ]:
final_df = new_df.rename(columns={"H_MEDIAN": "hourly_median", "A_MEDIAN": "annual_median", "OCC_CODE": "occupation_code", "OCC_TITLE": "occupation_title"})
final_df

In [ ]:
final_df.drop(columns=["ANNUAL", "HOURLY"])

In [ ]:
ca_df = final_df[final_df["ST"] == "CA"]
ca_df

In [ ]:
occupations = ca_df[["occupation_code", "occupation_title"]].copy()
occupations.reset_index(drop=True, inplace=True)
occupations

In [ ]:
yearly_stats = ca_df[["occupation_code", "hourly_median", "annual_median"]].copy()
yearly_stats.reset_index(drop=True, inplace=True)
yearly_stats

In [ ]:
type(yearly_stats.iloc[1,2])

In [ ]:
cols = ["hourly_median", "annual_median"]
yearly_stats[cols] = yearly_stats[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [ ]:
rds_connection_string = "root:root@127.0.0.1:3306/tech_occupational_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

In [ ]:
# Upload table 1 info
occupations.to_sql(name='occupations', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from occupations', con=engine).head()

In [ ]:
# Upload table 2 info
yearly_stats.to_sql(name='yearly_stats', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from yearly_stats', con=engine).head()